# TASK 1
Implement the encryptor for a simplified AES-like cipher with the parameters given in the previous slides and the following substitution function:
    $$f : y_j(j): 2v_i(j) \mod  p, \ j$$

In [157]:
import numpy as np

In [158]:
a=np.array([0,1,2])
b=np.array([3,4,5])

c = a+b

print(c)

[3 5 7]


In [159]:
def subkey_generator(key):

    k_1 = [key[0],key[2],key[4],key[6]] 
    k_2 = [key[0],key[1],key[2],key[6]] 
    k_3 = [key[0],key[3],key[4],key[6]] 
    k_4 = [key[0],key[3],key[5],key[6]] 
    k_5 = [key[0],key[2],key[5],key[6]] 
    k_6 = [key[2],key[4],key[4],key[6]] 

    return [k_1, k_2, k_3, k_4, k_5, k_6]

def subkey_sum(k_i,w,p):

    k_inter = k_i+k_i
    k = np.array(k_inter) #list

    return (w + k)%p



In [160]:
 
def timesTwo(v,p):

    return np.multiply(v,2)%p

def transposition(y):

    return np.array([y[0],y[1],y[2],y[3],y[7],y[6],y[5],y[4]])

def linear(z, p):

    z_matrix = z.reshape(2,4)

    param_matrix = np.matrix([[2,5],[1,7]])

    w_matrix = np.dot(param_matrix,z_matrix)%p

    w_array = np.asarray(w_matrix).flatten()

    #reordered_array = np.array([w_array[0], w_array[2], w_array[4], w_array[6], w_array[1], w_array[3], w_array[5], w_array[7]])

    return w_array
        




In [161]:
def encryption(u,k,p,round):

    k_list = subkey_generator(k)

    for i in range (round):
        
        print("round : ",i)
        v = subkey_sum(k_list[i],u,p)            #v = u+k
        print("post subkey_sum",v)
        y = timesTwo(v,p)
        print("post timesTwo",y)
        z = transposition(y)
        print("post transpo",z)
        if i == round-1:
            x = subkey_sum(k_list[i+1],z,p)
            break
        u = linear(z, p)
        print("post linear", u)
        
    return x


In [162]:
u = np.array([1,0,0,0,0,0,0,0])
k = np.array([1,0,0,0,0,0,0,0])
p = 11
r = 5

x = encryption(u,k,p,r)
print(x)


round :  0
post subkey_sum [2 0 0 0 1 0 0 0]
post timesTwo [4 0 0 0 2 0 0 0]
post transpo [4 0 0 0 0 0 0 2]
post linear [ 8  0  0 10  4  0  0  3]
round :  1
post subkey_sum [ 9  0  0 10  5  0  0  3]
post timesTwo [ 7  0  0  9 10  0  0  6]
post transpo [ 7  0  0  9  6  0  0 10]
post linear [0 0 0 2 5 0 0 2]
round :  2
post subkey_sum [1 0 0 2 6 0 0 2]
post timesTwo [2 0 0 4 1 0 0 4]
post transpo [2 0 0 4 4 0 0 1]
post linear [2 0 0 2 8 0 0 0]
round :  3
post subkey_sum [3 0 0 2 9 0 0 0]
post timesTwo [6 0 0 4 7 0 0 0]
post transpo [6 0 0 4 0 0 0 7]
post linear [ 1  0  0 10  6  0  0  9]
round :  4
post subkey_sum [ 2  0  0 10  7  0  0  9]
post timesTwo [4 0 0 9 3 0 0 7]
post transpo [4 0 0 9 7 0 0 3]
[4 0 0 9 7 0 0 3]


# TASK 2
Implement the decryptor for this simplified AES-like cipher. Note that decryption is performed by the inverse blocks in reverse order. Therefore, you have to implement the inverse of each function used to encrypt the message (subkey sum, substitution, transposition and linear), taking into consideration that all the operations must be done in the field $\mathbb{F} = GF(p)$.

In [163]:
def inv_timesTwo(v,p):
    mul_inv = pow(2,-1,p)
    return np.multiply(v,mul_inv)%p


def inv_transposition(y):
    return np.array([y[0],y[1],y[2],y[3],y[7],y[6],y[5],y[4]])


def inv_subkey_sum(k_i,w,p):

    k_inter = k_i+k_i
    k = np.array(k_inter) #list

    return (w - k)%p


def inv_linear(z, p):
    z_matrix = z.reshape(2, 4)

    param_matrix = np.array([[2, 5], [1, 7]])

    #Calcolo dell'inversa della matrice modulo p
    det = int(np.round(np.linalg.det(param_matrix)))  #Determinante della matrice
    det_inv = pow(det, -1, p)  #Inverso del determinante modulo p
    adj_matrix = np.array([[param_matrix[1, 1], -param_matrix[0, 1]],
                           [-param_matrix[1, 0], param_matrix[0, 0]]])  #Matrice aggiunta
    param_matrix_inv = (det_inv * adj_matrix) % p  #Inversa modulo p

    w_matrix = np.dot(param_matrix_inv, z_matrix) % p

    w_array = np.asarray(w_matrix, dtype=int).flatten()

    return w_array


def decryption(x,k,p,round):
    k_list = subkey_generator(k)

    for i in range(round,-1,-1):
        print("round : ", i)

        z = inv_subkey_sum(k_list[i],x,p)            #v = u - k
        print("post subkey_sum",z)
        
        if i == 0:
            x = z
            break
        elif i != round:
            w = inv_linear(z, p)
            print("post linear", w)
        else:
            w = z

        y = inv_transposition(w)
        print("post transpo",y)

        x = inv_timesTwo(y,p)
        print("post timesTwo",x)
        
    return x

Test for inv_linear

In [164]:
a = linear(np.array([1,0,3,5,0,0,8,0]), p)  #change the array to test
print(a)
b = inv_linear(a, p)
print(b)

[ 2  0  2 10  1  0  4  5]
[1 0 3 5 0 0 8 0]


In [165]:
u = decryption(x,k,p,r)
print(u)

round :  5
post subkey_sum [4 0 0 9 7 0 0 3]
post transpo [4 0 0 9 3 0 0 7]
post timesTwo [ 2  0  0 10  7  0  0  9]
round :  4
post subkey_sum [ 1  0  0 10  6  0  0  9]
post linear [6 0 0 4 0 0 0 7]
post transpo [6 0 0 4 7 0 0 0]
post timesTwo [3 0 0 2 9 0 0 0]
round :  3
post subkey_sum [2 0 0 2 8 0 0 0]
post linear [2 0 0 4 4 0 0 1]
post transpo [2 0 0 4 1 0 0 4]
post timesTwo [1 0 0 2 6 0 0 2]
round :  2
post subkey_sum [0 0 0 2 5 0 0 2]
post linear [ 7  0  0  9  6  0  0 10]
post transpo [ 7  0  0  9 10  0  0  6]
post timesTwo [ 9  0  0 10  5  0  0  3]
round :  1
post subkey_sum [ 8  0  0 10  4  0  0  3]
post linear [4 0 0 0 0 0 0 2]
post transpo [4 0 0 0 2 0 0 0]
post timesTwo [2 0 0 0 1 0 0 0]
round :  0
post subkey_sum [1 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0]


Other tests for decryption

In [166]:
a = np.array([1,8,3,6,5,6,7,2]) #change plaintext to test
k = np.array([1,0,4,0,3,0,2,0]) #change key to test
b = encryption(a,k,p,r)
print(b)
c = decryption(b,k,p,r)
print(c)

round :  0
post subkey_sum [ 2  1  6  8  6 10 10  4]
post timesTwo [4 2 1 5 1 9 9 8]
post transpo [4 2 1 5 8 9 9 1]
post linear [ 4  5  3  4  5 10  9  1]
round :  1
post subkey_sum [ 5  5  7  6  6 10  2  3]
post timesTwo [10 10  3  1  1  9  4  6]
post transpo [10 10  3  1  6  4  9  1]
post linear [6 7 7 7 8 5 0 8]
round :  2
post subkey_sum [ 7  7 10  9  9  5  3 10]
post timesTwo [ 3  3  9  7  7 10  6  9]
post transpo [ 3  3  9  7  9  6 10  7]
post linear [7 3 2 5 0 1 2 1]
round :  3
post subkey_sum [8 3 2 7 1 1 2 3]
post timesTwo [5 6 4 3 2 2 4 6]
post transpo [5 6 4 3 6 4 2 2]
post linear [ 7 10  7  5  3  1  7  6]
round :  4
post subkey_sum [8 3 7 7 4 5 7 8]
post timesTwo [ 5  6  3  3  8 10  3  5]
post transpo [ 5  6  3  3  5  3 10  8]
[ 9  9  6  5  9  6  2 10]
round :  5
post subkey_sum [ 5  6  3  3  5  3 10  8]
post transpo [ 5  6  3  3  8 10  3  5]
post timesTwo [8 3 7 7 4 5 7 8]
round :  4
post subkey_sum [ 7 10  7  5  3  1  7  6]
post linear [5 6 4 3 6 4 2 2]
post transpo [5 6 4